In [1]:
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter

%matplotlib inline

In [2]:
path_to_df = '../data/Book/ratings_Books.csv'
df = pd.read_csv(path_to_df, names=['user_id', 'item_id', 'rating', 'timestamp'])
print(df.shape)

(22507155, 4)


In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,AH2L9G3DQHHAJ,0000000116,4.0,1019865600
1,A2IIIDRK3PRRZY,0000000116,1.0,1395619200
2,A1TADCM7YWPQ8M,0000000868,4.0,1031702400
3,AWGH7V0BDOJKB,0000013714,4.0,1383177600
4,A3UTQPQPM4TQO0,0000013714,5.0,1374883200


In [4]:
df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [5]:
df.user_id.max(), df.user_id.unique().shape

('AZZZZCVMZCKUJ', (8026324,))

In [6]:
df.user_id = pd.factorize(df.user_id)[0] + 1
df.user_id.min(), df.user_id.max(), df.user_id.unique().shape

(1, 8026324, (8026324,))

In [7]:
df.item_id = pd.factorize(df.item_id)[0] + 1
df.item_id.min(), df.item_id.max(), df.item_id.unique().shape

(1, 2330066, (2330066,))

In [8]:
df.head()

,user_id,item_id,rating,timestamp
0,1,1,4.0,1019865600
1,2,1,1.0,1395619200
2,3,2,4.0,1031702400
3,4,3,4.0,1383177600
4,5,3,5.0,1374883200


In [9]:
sorted_timestamps = sorted(df.timestamp)
len(sorted_timestamps)

22507155

In [10]:
threshold_timestamp = sorted_timestamps[int(len(sorted_timestamps) * (1.0 - 0.2))]
test_threshold_timestamp = sorted_timestamps[int(len(sorted_timestamps) * (1.0 - 0.1))]

print(threshold_timestamp, test_threshold_timestamp)

1389225600 1397347200


In [11]:
train_data = df[df.timestamp < threshold_timestamp]
validation_data = df[(threshold_timestamp <= df.timestamp) & (df.timestamp < test_threshold_timestamp)]
test_data = df[test_threshold_timestamp <= df.timestamp]

train_data.shape, validation_data.shape, test_data.shape

((17980657, 4), (2270889, 4), (2255609, 4))

In [12]:
data = []

for _, row in tqdm(train_data.iterrows()):
    data.append({
        'user_id': int(row.user_id),
        'item_id': int(row.item_id),
        'timestamp': int(row.timestamp)
    })

print(len(data))

17980657it [13:19, 22501.35it/s]

17980657


In [13]:
user_history = defaultdict(list)
item_history = defaultdict(list)

for row in tqdm(data):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    
    user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
    item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


is_changed = True
threshold = 4
good_users = set()
good_items = set()


while is_changed:
    old_state = (len(good_users), len(good_items))
    
    good_users = set()
    good_items = set()

    for user_id, history in user_history.items():
        if len(history) >= threshold:
            good_users.add(user_id)

    for item_id, history in item_history.items():
        if len(history) >= threshold:
            good_items.add(item_id)
    
    user_history = {
        user_id: list(filter(lambda x: x['item_id'] in good_items, history))
        for user_id, history in user_history.items()
    }
    
    item_history = {
        item_id: list(filter(lambda x: x['user_id'] in good_users, history))
        for item_id, history in item_history.items()
    }
    
    new_state = (len(good_users), len(good_items))
    is_changed = (old_state != new_state)
    print(old_state, new_state)

100%|██████████████████████████████████████████████████████████████████| 17980657/17980657 [01:11<00:00, 250206.00it/s]


(0, 0) (868064, 757306)
(868064, 757306) (751874, 452368)
(751874, 452368) (683028, 430958)
(683028, 430958) (675301, 415065)
(675301, 415065) (668443, 412933)
(668443, 412933) (667495, 411029)
(667495, 411029) (666593, 410726)
(666593, 410726) (666443, 410435)
(666443, 410435) (666281, 410393)
(666281, 410393) (666261, 410347)
(666261, 410347) (666241, 410344)
(666241, 410344) (666240, 410335)
(666240, 410335) (666235, 410333)
(666235, 410333) (666232, 410329)
(666232, 410329) (666230, 410328)
(666230, 410328) (666230, 410327)
(666230, 410327) (666228, 410327)
(666228, 410327) (666228, 410325)
(666228, 410325) (666226, 410325)
(666226, 410325) (666226, 410323)
(666226, 410323) (666225, 410323)
(666225, 410323) (666225, 410322)
(666225, 410322) (666223, 410322)
(666223, 410322) (666223, 410319)
(666223, 410319) (666222, 410319)
(666222, 410319) (666222, 410318)
(666222, 410318) (666222, 410318)


In [14]:
user_mapping = {}
item_mapping = {}
tmp_user_history = defaultdict(list)
tmp_item_history = defaultdict(list)

for raw_user_id, history in tqdm(user_history.items()):
    if len(history) >= threshold:
        processed_history = []

        for filtered_item_event in history:
            raw_item_id = filtered_item_event['item_id']
            item_timestamp = filtered_item_event['timestamp']

            processed_item_id = item_mapping.get(raw_item_id, len(item_mapping) + 1)
            item_mapping[raw_item_id] = processed_item_id

            processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
        processed_user_id = user_mapping.get(raw_user_id, len(user_mapping) + 1)
        user_mapping[raw_user_id] = processed_user_id

        tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    
for raw_item_id, history in tqdm(item_history.items()):
    if len(history) >= threshold:
        processed_history = []

        for filtered_user_event in history:
            raw_user_id = filtered_user_event['user_id']
            user_timestamp = filtered_user_event['timestamp']

            processed_user_id = user_mapping.get(raw_user_id, len(user_mapping) + 1)
            user_mapping[raw_user_id] = processed_user_id

            processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})


        processed_item_id = item_mapping.get(raw_item_id, len(item_mapping) + 1)
        item_mapping[raw_item_id] = processed_item_id

        tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

train_user_history = tmp_user_history
train_item_history = tmp_item_history

100%|█████████████████████████████████████████████████████████████████████| 2088849/2088849 [00:50<00:00, 40975.77it/s]


In [15]:
print('Users count:', len(user_mapping))
print('Items count:', len(item_mapping))
print('Actions count:', sum(list(map(lambda x: len(x), user_history.values()))))
print('Avg train user history len:', np.mean(list(map(lambda x: len(x), train_user_history.values()))))
print('Avg train item history len:', np.mean(list(map(lambda x: len(x), train_user_history.values()))))

Users count: 666222
Items count: 410318
Actions count: 13654461
Avg train user history len: 11.65730492238323
Avg train item history len: 11.65730492238323


In [16]:
train_data_filtered = train_data[train_data.user_id.isin(user_mapping) & train_data.item_id.isin(item_mapping)]
validation_data_filtered = validation_data[validation_data.user_id.isin(user_mapping) & validation_data.item_id.isin(item_mapping)]
test_data_filtered = test_data[test_data.user_id.isin(user_mapping) & test_data.item_id.isin(item_mapping)]

print(f'Train data. Before: {train_data.shape}. After: {train_data_filtered.shape}')
print(f'Validation data. Before: {validation_data.shape}. After: {validation_data_filtered.shape}')
print(f'Test data. Before: {test_data.shape}. After: {test_data_filtered.shape}')

Train data. Before: (17980657, 4). After: (7766353, 4)
Validation data. Before: (2270889, 4). After: (471944, 4)
Test data. Before: (2255609, 4). After: (329187, 4)


In [17]:
# Save train data (TODO do we need to duplicate: I think yes)
with open('../data/Book/train_new.txt', 'w') as f:
    cnt = 0
    cnt_added = 0
    for user_id, history in train_user_history.items():
        previous_history = []
        
        for item_event in sorted(history, key=lambda x: x['timestamp']):
            if len(previous_history) + 1 >= threshold:
                f.write(' '.join([str(user_id)] + previous_history + [str(item_event['item_id'])]))
                f.write('\n')
                cnt_added += 1
            
            previous_history.append(str(item_event['item_id']))
            cnt += 1
            
    assert cnt == train_data_filtered.shape[0]
    assert cnt == cnt_added + (len(user_mapping) * (threshold - 1))

In [18]:
validation_list = []

for _, row in tqdm(validation_data_filtered.iterrows()):
    validation_list.append({
        'user_id': int(row.user_id),
        'item_id': int(row.item_id),
        'timestamp': int(row.timestamp)
    })
validation_list = sorted(validation_list, key=lambda x: x['timestamp'])
print(len(validation_list))


# Add events to `validation_user_history`
validation_user_history = defaultdict(list)
for row in tqdm(validation_list):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    validation_user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})

# Re-number user and item IDs
tmp_user_history = defaultdict(list)
for user_id, history in tqdm(validation_user_history.items()):
    processed_user_id = user_mapping[user_id]
    
    processed_history = []
    for item_event in history:
        item_id = item_event['item_id']
        item_timestamp = item_event['timestamp']
        processed_item_id = item_mapping[item_id]
        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])
validation_user_history = tmp_user_history

471944it [00:21, 21799.81it/s]
 28%|███████████████████▍                                                  | 131120/471944 [00:00<00:00, 667617.10it/s]

471944


100%|███████████████████████████████████████████████████████████████████████| 156088/156088 [00:02<00:00, 75332.24it/s]


In [19]:
# Save validation data
with open('../data/Book/validation_new.txt', 'w') as f:
    cnt = 0
    for user_id, validation_history in validation_user_history.items():
        previous_history = [
            str(event['item_id']) for event in sorted(train_user_history[user_id], key=lambda x: x['timestamp'])
        ]
        assert len(previous_history) > 0
        
        for validation_item_event in  sorted(validation_history, key=lambda x: x['timestamp']):
            assert len(previous_history) + 1 >= threshold
            
            f.write(' '.join([str(user_id)] + previous_history + [str(validation_item_event['item_id'])]))
            f.write('\n')
            
            previous_history.append(str(validation_item_event['item_id']))
            cnt += 1
            
    assert cnt == validation_data_filtered.shape[0]

In [20]:
test_list = []

for _, row in tqdm(test_data_filtered.iterrows()):
    test_list.append({
        'user_id': int(row.user_id),
        'item_id': int(row.item_id),
        'timestamp': int(row.timestamp)
    })
test_list = sorted(test_list, key=lambda x: x['timestamp'])
print(len(test_list))

# Add events to `test_user_history`
test_user_history = defaultdict(list)
for row in tqdm(test_list):
    user_raw_id = row['user_id']
    item_raw_id = row['item_id']
    interaction_timestamp = row['timestamp']
    test_user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})

# Re-number user and item IDs
tmp_user_history = defaultdict(list)
for user_id, history in tqdm(test_user_history.items()):
    processed_user_id = user_mapping[user_id]
    
    processed_history = []
    for item_event in history:
        item_id = item_event['item_id']
        item_timestamp = item_event['timestamp']
        processed_item_id = item_mapping[item_id]
        processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})
        
    tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])
test_user_history = tmp_user_history

329187it [00:15, 21770.19it/s]
 47%|████████████████████████████████▋                                     | 153537/329187 [00:00<00:00, 771494.22it/s]

329187


100%|██████████████████████████████████████████████████████████████████████| 124953/124953 [00:00<00:00, 191352.54it/s]


In [21]:
# Save test data
with open('../data/Book/test_new.txt', 'w') as f:
    cnt = 0
    for user_id, test_history in test_user_history.items():
        train_history = [
            str(event['item_id']) for event in sorted(train_user_history[user_id], key=lambda x: x['timestamp'])
        ]
        validation_history = [
            str(event['item_id']) for event in sorted(validation_user_history[user_id], key=lambda x: x['timestamp'])
        ]
        previous_history = train_history + validation_history
        assert len(train_history) > 0
        
        for test_item_event in sorted(test_history, key=lambda x: x['timestamp']):
            f.write(' '.join([str(user_id)] + previous_history + [str(test_item_event['item_id'])]))
            f.write('\n')
            
            previous_history.append(str(test_item_event['item_id']))
            cnt += 1
            
    assert cnt == test_data_filtered.shape[0]